#  Grid4x4

In [1]:
import os

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecMonitor
from sumo_rl import parallel_env
import supersuit as ss

from evaluate import evaluate

In [2]:
TEST_NUM = 1

## Create Environment and Instantiate Agent

In [3]:
from reward_functions import diff_wait_time

env_params = {
    "net_file": os.path.join("nets","RESCO","grid4x4","grid4x4.net.xml"),
    "route_file": os.path.join("nets","RESCO","grid4x4","grid4x4_1.rou.xml"),
    "num_seconds": 3600,
    "reward_fn": diff_wait_time,
    "sumo_seed": 42,
}
env = parallel_env(**env_params)

# Maybe add frame-stacking here
env = ss.pettingzoo_env_to_vec_env_v1(env)
env = ss.concat_vec_envs_v1(env, num_vec_envs=2, num_cpus=1, base_class="stable_baselines3")
env = VecMonitor(env)

In [4]:
# Using hyperparams from RESCO supplement/appendix Table 5
# https://datasets-benchmarks-proceedings.neurips.cc/paper/2021/hash/f0935e4cd5920aa6c7c996a5ee53a70f-Abstract-round1.html

model = PPO(
    "MlpPolicy",
    env,
    learning_rate=2.5e-4,
    gamma=0.99,
    ent_coef=1e-3,
    max_grad_norm=0.5,
    n_steps=1024,
    batch_size=256,
    n_epochs=4,
    gae_lambda=0.95,
    clip_range=0.1,
    # vf_coef=1.0,
    tensorboard_log=os.path.join("logs","grid4x4"),
    verbose=1
)

Using cuda device


In [5]:
# Evaluate untrained random agent
csv_dir = os.path.join("outputs","grid4x4",f"test_{TEST_NUM}")
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)

csv_path = os.path.join(csv_dir, "untrained.csv")
tb_log_dir = os.path.join("logs","grid4x4","eval_untrained")

mean_reward_untrained, std_reward_untrained = evaluate(model, env, csv_path, tb_log_dir, n_eval_episodes=1)

# Change made to SB3 > common > vec_env > vec_monitor.py > VecMonitor
# Line 76 - added extra item to self.venv.step_wait() return
# obs, rewards, dones, infos --> obs, rewards, dones, _, infos

print(mean_reward_untrained)
print(std_reward_untrained)

-205261.0
0.0


## Train and Evaluate Agent

In [6]:
model.learn(total_timesteps=1e5)

Logging to logs\grid4x4\PPO_2
------------------------------
| time/              |       |
|    fps             | 195   |
|    iterations      | 1     |
|    time_elapsed    | 167   |
|    total_timesteps | 32768 |
------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 190          |
|    iterations           | 2            |
|    time_elapsed         | 344          |
|    total_timesteps      | 65536        |
| train/                  |              |
|    approx_kl            | 0.0016727817 |
|    clip_fraction        | 0.0336       |
|    clip_range           | 0.1          |
|    entropy_loss         | -2.08        |
|    explained_variance   | 1.13e-05     |
|    learning_rate        | 0.00025      |
|    loss                 | 9.65e+03     |
|    n_updates            | 4            |
|    policy_gradient_loss | -0.00237     |
|    value_loss           | 6.85e+03     |
----------------------

In [7]:
# Evaluate trained agent
csv_path = os.path.join(csv_dir, "trained.csv")
tb_log_dir = os.path.join("logs","grid4x4",f"PPO_{TEST_NUM}","eval_trained")

mean_reward_trained, std_reward_trained = evaluate(model, env, csv_path, tb_log_dir, n_eval_episodes=1)

print(mean_reward_trained)
print(std_reward_trained)

-80570.0
0.0


In [8]:
# Move untrained.csv into numbered directory
import shutil

original = os.path.join("logs","grid4x4","eval_untrained")
target = os.path.join("logs","grid4x4",f"PPO_{TEST_NUM}","eval_untrained")

shutil.move(original, target)

'logs\\grid4x4\\PPO_1\\eval_untrained'

### Save Model

In [17]:
model.save(os.path.join("saved_models",f"PPO_grid4x4_{TEST_NUM}"))

## Process Results

In [16]:
import csv

data = [env_params["net_file"], TEST_NUM, env_params["reward_fn"].__name__, mean_reward_untrained, mean_reward_trained]

with open("test_results.csv", "a", encoding="UTF8", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(data)

### Benchmark Comparison

In [ ]:
import pandas as pd

results = pd.read_csv()

## Render Video

In [47]:
env_params.update({"use_gui": True, "render_mode": "human"})
render_env = parallel_env(**env_params)

# Maybe add frame-stacking here
render_env = ss.pettingzoo_env_to_vec_env_v1(render_env)
render_env = VecMonitor(render_env)

In [48]:
import pyautogui

obs = render_env.reset()

folder_path = os.path.join("renders","grid4x4",f"test_{TEST_NUM}")
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

max_time = env.unwrapped.par_env.unwrapped.env.sim_max_time
delta_time = env.unwrapped.par_env.unwrapped.env.delta_time
vid_length = round(max_time/delta_time)

for i in range(vid_length):
    actions, _ = model.predict(obs)
    obs, rewards, dones, infos = render_env.step(actions)
    render_env.render()

    im = pyautogui.screenshot(
        os.path.join(folder_path,f"img{i}.jpg"),
        region=(0, 0, 2560, 1542)
    )

render_env.close()  # clean up

TypeError: render() got an unexpected keyword argument 'mode'

In [58]:
render_env.unwrapped.par_env.unwrapped.env.virtual_display

(3200, 1800)

In [ ]:
import subprocess

subprocess.run([
    "ffmpeg", "-y", "-r", "5", "-i",
    f"renders\\grid4x4\\test_{TEST_NUM}\\img%d.jpg",
    f"videos\\grid4x4_{TEST_NUM}.mp4"
])

## Clean-up

In [59]:
env.close()
render_env.close()